# kjv gpt model

In [2]:
# starting with a data set to train on
!wget https://https://www.o-bible.com/download/kjv.txt 

zsh:1: command not found: wget


In [3]:
# read it and inspect it
with open('kjv.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  4404381


In [5]:
# let's look at the first 1000 characters
print(text[:1000])

Ge1:1 In the beginning God created the heaven and the earth.
Ge1:2 And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters.
Ge1:3 And God said, Let there be light: and there was light.
Ge1:4 And God saw the light, that it was good: and God divided the light from the darkness.
Ge1:5 And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day.
Ge1:6 And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters.
Ge1:7 And God made the firmament, and divided the waters which were under the firmament from the waters which were above the firmament: and it was so.
Ge1:8 And God called the firmament Heaven. And the evening and the morning were the second day.
Ge1:9 And God said, Let the waters under the heaven be gathered together unto one place, and let the dry land appear: and it was so.
Ge1:10 And God c

In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz
73


In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hello world"))
print(decode(encode("hello world")))

[54, 51, 58, 58, 61, 1, 69, 61, 64, 58, 50]
hello world


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earlier will look like this to GPT

torch.Size([4404381]) torch.int64
tensor([28, 51, 10, 19, 10,  1, 30, 60,  1, 66, 54, 51,  1, 48, 51, 53, 55, 60,
        60, 55, 60, 53,  1, 28, 61, 50,  1, 49, 64, 51, 47, 66, 51, 50,  1, 66,
        54, 51,  1, 54, 51, 47, 68, 51, 60,  1, 47, 60, 50,  1, 66, 54, 51,  1,
        51, 47, 64, 66, 54,  8,  0, 28, 51, 10, 19, 11,  1, 22, 60, 50,  1, 66,
        54, 51,  1, 51, 47, 64, 66, 54,  1, 69, 47, 65,  1, 69, 55, 66, 54, 61,
        67, 66,  1, 52, 61, 64, 59,  6,  1, 47, 60, 50,  1, 68, 61, 55, 50, 20,
         1, 47, 60, 50,  1, 50, 47, 64, 57, 60, 51, 65, 65,  1, 69, 47, 65,  1,
        67, 62, 61, 60,  1, 66, 54, 51,  1, 52, 47, 49, 51,  1, 61, 52,  1, 66,
        54, 51,  1, 50, 51, 51, 62,  8,  1, 22, 60, 50,  1, 66, 54, 51,  1, 40,
        62, 55, 64, 55, 66,  1, 61, 52,  1, 28, 61, 50,  1, 59, 61, 68, 51, 50,
         1, 67, 62, 61, 60,  1, 66, 54, 51,  1, 52, 47, 49, 51,  1, 61, 52,  1,
        66, 54, 51,  1, 69, 47, 66, 51, 64, 65,  8,  0, 28, 51, 10, 19, 12,  1,
      

In [9]:
# let's now split up the data into training and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size + 1]

tensor([28, 51, 10, 19, 10,  1, 30, 60,  1])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([28]) the target: 51
when input is tensor([28, 51]) the target: 10
when input is tensor([28, 51, 10]) the target: 19
when input is tensor([28, 51, 10, 19]) the target: 10
when input is tensor([28, 51, 10, 19, 10]) the target: 1
when input is tensor([28, 51, 10, 19, 10,  1]) the target: 30
when input is tensor([28, 51, 10, 19, 10,  1, 30]) the target: 60
when input is tensor([28, 51, 10, 19, 10,  1, 30, 60]) the target: 1


In [12]:
torch.manual_seed(1111)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1, 49, 61, 64, 50,  1, 69, 54],
        [58, 51, 59,  6,  1, 31, 51, 64],
        [67, 66,  1, 66, 54, 55, 65,  1],
        [ 8,  0, 26, 72, 51, 13, 14, 19]])
targets:
torch.Size([4, 8])
tensor([[49, 61, 64, 50,  1, 69, 54, 55],
        [51, 59,  6,  1, 31, 51, 64, 67],
        [66,  1, 66, 54, 55, 65,  1, 61],
        [ 0, 26, 72, 51, 13, 14, 19, 10]])
----
when input is [1] the target: 49
when input is [1, 49] the target: 61
when input is [1, 49, 61] the target: 64
when input is [1, 49, 61, 64] the target: 50
when input is [1, 49, 61, 64, 50] the target: 1
when input is [1, 49, 61, 64, 50, 1] the target: 69
when input is [1, 49, 61, 64, 50, 1, 69] the target: 54
when input is [1, 49, 61, 64, 50, 1, 69, 54] the target: 55
when input is [58] the target: 51
when input is [58, 51] the target: 59
when input is [58, 51, 59] the target: 6
when input is [58, 51, 59, 6] the target: 1
when input is [58, 51, 59, 6, 1] the target: 31
when input is [58, 51, 59

In [13]:
print(xb) # our input to the transformer

tensor([[ 1, 49, 61, 64, 50,  1, 69, 54],
        [58, 51, 59,  6,  1, 31, 51, 64],
        [67, 66,  1, 66, 54, 55, 65,  1],
        [ 8,  0, 26, 72, 51, 13, 14, 19]])


In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1111)

class BigramLanguagemodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else: 
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # forus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguagemodel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 73])
tensor(4.9410, grad_fn=<NllLossBackward0>)

M4eGPCyeuRYgj?NHM0?IrNcz4DHTBe;Vu2?1ohmDhycfnA:KRg0r'B2o:Ol)b7ahbJV2PS
O8V1v!1)0Ax?3c
IeoHnM9LRbd;DF


In [15]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [16]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')
    #evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.402892827987671


In [17]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=120)[0].tolist()))


Gait sreat ponh s grkif mere ofe ccte senoner TLe m, he wiPriond s aire y f Istof the thone ain f u Gether22:3 an r atec


# The mathematical trick in self-attention

In [18]:
# consider the following toy example:

torch.manual_seed(1111)
B, T, C = 4,8,2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [19]:
# Version 1

# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # xbow is x bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [20]:
# Version 2

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B,T,T) @ (B,T,C) ====> (B,T,C)
torch.allclose(xbow, xbow2)

True

In [21]:
# matrix multiplication explanation

torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a,1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [22]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


NameError: name 'tril' is not defined